In [6]:
import requests
import pandas as pd
import h3

# Define the Overpass API endpoint and query
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["name"="Chicago"]->.searchArea;
(
  node["amenity"="restaurant"](area.searchArea);
  node["amenity"="bar"](area.searchArea);
  node["amenity"="cafe"](area.searchArea);
  node["amenity"="hotel"](area.searchArea);
  node["amenity"="hospital"](area.searchArea);
  node["amenity"="airport"](area.searchArea);
  node["railway"="station"](area.searchArea);
  node["amenity"="public_transport"](area.searchArea);
);
out body;
"""

# Send the query to the Overpass API
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

In [7]:
# Extract relevant data from the response
pois = []
for element in data['elements']:
    tags = element.get('tags', {})
    poi_type = (tags.get('amenity') or tags.get('leisure') or 
                    tags.get('tourism') or tags.get('railway'))
    poi = {
        'id': element['id'],
        'lat': element['lat'],
        'lon': element['lon'],
        'name': tags.get('name'),
        "poi_type": poi_type
    }
    pois.append(poi)

# Create a DataFrame from the list of POIs
df = pd.DataFrame(pois)
df

,id,lat,lon,name,poi_type
0,252643581,42.008086,-87.665969,Morse,station
1,252643632,41.953887,-87.654928,Sheridan,station
2,252683058,41.947520,-87.653627,Addison,station
3,252683064,41.939796,-87.653328,Belmont,station
4,252683073,41.918242,-87.652631,Armitage,station
...,...,...,...,...,...
3591,12007620701,41.886966,-87.648427,Bar La Rue,restaurant
3592,12007620702,41.887063,-87.648467,La Serre,restaurant
3593,12007620703,41.887648,-87.648441,Sunda,restaurant
3594,12012367898,41.885924,-87.625186,Roti,restaurant


In [8]:
import seaborn as sns
df["poi_type"].value_counts()

poi_type
restaurant    1950
bar            760
cafe           700
station        186
Name: count, dtype: int64

In [9]:
def add_h3_location(row, resolution):
    return h3.geo_to_h3(row['lat'], row['lon'], resolution)

resolutions = [4,6,8]

# H3-Location-Spalte hinzufügen
for res in resolutions:
    df[f"h3_res_{res}"] = df.apply(lambda row: add_h3_location(row, res), axis=1)
df

,id,lat,lon,name,poi_type,h3_res_4,h3_res_6,h3_res_8
0,252643581,42.008086,-87.665969,Morse,station,842664dffffffff,862664d8fffffff,882664d8c1fffff
1,252643632,41.953887,-87.654928,Sheridan,station,842664dffffffff,862664c17ffffff,882664c169fffff
2,252683058,41.947520,-87.653627,Addison,station,842664dffffffff,862664c17ffffff,882664c16bfffff
3,252683064,41.939796,-87.653328,Belmont,station,842664dffffffff,862664c17ffffff,882664c163fffff
4,252683073,41.918242,-87.652631,Armitage,station,842664dffffffff,862664c17ffffff,882664c13dfffff
...,...,...,...,...,...,...,...,...
3591,12007620701,41.886966,-87.648427,Bar La Rue,restaurant,842664dffffffff,862664cafffffff,882664cad3fffff
3592,12007620702,41.887063,-87.648467,La Serre,restaurant,842664dffffffff,862664cafffffff,882664cad3fffff
3593,12007620703,41.887648,-87.648441,Sunda,restaurant,842664dffffffff,862664cafffffff,882664cadbfffff
3594,12012367898,41.885924,-87.625186,Roti,restaurant,842664dffffffff,862664c1fffffff,882664c1e3fffff


In [10]:
df.to_csv("poi_data.csv",index=False)